# Notebook 1: MCP Foundations & Architecture

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 1 of 13  
**Level:** Beginner  
**Duration:** ~45 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Understand what the Model Context Protocol (MCP) is and why it matters
2. Grasp the M×N interoperability problem and how MCP solves it
3. Learn the Client-Host-Server architectural triad
4. Understand MCP's core primitives (Tools, Resources, Prompts)
5. Know the difference between STDIO and HTTP transports
6. Set up your development environment for MCP

## 1. Introduction: The Shift to Agentic AI

The deployment of Large Language Models (LLMs) is undergoing a fundamental architectural transition:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                        THE EVOLUTION OF LLM USAGE                        │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   PHASE 1: Chat Interfaces          PHASE 2: Agentic Workflows          │
│   ─────────────────────             ───────────────────────────          │
│                                                                          │
│   ┌─────────┐                       ┌─────────┐                         │
│   │  User   │                       │  User   │                         │
│   └────┬────┘                       └────┬────┘                         │
│        │                                 │                              │
│        ▼                                 ▼                              │
│   ┌─────────┐                       ┌─────────┐      ┌──────────┐      │
│   │   LLM   │                       │  Agent  │◄────►│ Database │      │
│   └─────────┘                       │  (LLM)  │      └──────────┘      │
│        │                            │         │      ┌──────────┐      │
│        ▼                            │         │◄────►│   APIs   │      │
│   ┌─────────┐                       │         │      └──────────┘      │
│   │ Response│                       │         │      ┌──────────┐      │
│   └─────────┘                       │         │◄────►│  Files   │      │
│                                     └─────────┘      └──────────┘      │
│                                                                          │
│   • Isolated                        • Integrated                        │
│   • Ephemeral                       • Persistent                        │
│   • Text-only                       • Action-capable                    │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

**Agentic AI** refers to AI systems that can:
- **Plan** multi-step tasks autonomously
- **Reason** about context and make decisions
- **Act** on external systems (databases, APIs, files)
- **Persist** state across interactions

## 2. The M×N Interoperability Problem

### The Challenge

Before MCP, connecting LLMs to external tools created a **combinatorial explosion** of integration efforts:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    THE M×N INTEGRATION NIGHTMARE                         │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   MODELS (M)                              TOOLS (N)                      │
│   ──────────                              ─────────                      │
│                                                                          │
│   ┌──────────┐         Custom            ┌──────────┐                   │
│   │  Claude  │◄───────Adapter 1─────────►│ Postgres │                   │
│   └──────────┘◄───────Adapter 2─────────►├──────────┤                   │
│                ◄───────Adapter 3─────────►│  Slack   │                   │
│   ┌──────────┐◄───────Adapter 4─────────►├──────────┤                   │
│   │  GPT-4   │◄───────Adapter 5─────────►│  GitHub  │                   │
│   └──────────┘◄───────Adapter 6─────────►├──────────┤                   │
│                ◄───────Adapter 7─────────►│  Gmail   │                   │
│   ┌──────────┐◄───────Adapter 8─────────►├──────────┤                   │
│   │  Gemini  │◄───────Adapter 9─────────►│ Salesforce│                   │
│   └──────────┘◄──────Adapter 10─────────►└──────────┘                   │
│                        ...                                              │
│                                                                          │
│   3 Models × 5 Tools = 15 Custom Adapters!                              │
│   10 Models × 20 Tools = 200 Custom Adapters!                           │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

**Problems with M×N:**
- Each model needs a custom adapter for each tool
- Different authentication schemes per integration
- Different schema mappings and error handling
- Maintenance costs exceed integration value
- Fragile, unmaintainable codebases

### The MCP Solution: M + N

MCP introduces a **universal protocol** that reduces complexity from M×N to M+N:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP: THE UNIVERSAL INTERFACE                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   MODELS (M)                MCP                 TOOLS (N)               │
│   ──────────              PROTOCOL              ─────────               │
│                                                                          │
│   ┌──────────┐         ┌─────────┐         ┌──────────┐                │
│   │  Claude  │◄───────►│         │◄───────►│ Postgres │                │
│   └──────────┘         │         │         └──────────┘                │
│                        │         │                                      │
│   ┌──────────┐         │   MCP   │         ┌──────────┐                │
│   │  GPT-4   │◄───────►│ Standard│◄───────►│  Slack   │                │
│   └──────────┘         │         │         └──────────┘                │
│                        │         │                                      │
│   ┌──────────┐         │         │         ┌──────────┐                │
│   │  Gemini  │◄───────►│         │◄───────►│  GitHub  │                │
│   └──────────┘         └─────────┘         └──────────┘                │
│                                                                          │
│   3 Models + 3 Tools = 6 Implementations (not 9!)                       │
│   10 Models + 20 Tools = 30 Implementations (not 200!)                  │
│                                                                          │
│   ════════════════════════════════════════════════════                  │
│   MCP is the "USB-C for AI" - Write once, use everywhere                │
│   ════════════════════════════════════════════════════                  │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 3. What is MCP?

The **Model Context Protocol (MCP)** is an open-source client-server protocol that standardizes how AI models (clients) interact with external tools and data (servers).

### Key Characteristics

| Aspect | Description |
|--------|-------------|
| **Type** | Open-source protocol |
| **Communication** | JSON-RPC 2.0 based |
| **State** | Stateful sessions with capability negotiation |
| **Inspiration** | Language Server Protocol (LSP) |
| **Creator** | Anthropic (open-sourced Nov 2024) |

### What MCP Does

1. **Standardizes interfaces** - Common way for AI to interact with tools
2. **Manages context** - Structures external data for LLM consumption
3. **Enables capability negotiation** - Client and server agree on features
4. **Provides security boundaries** - Controlled access to resources
5. **Supports streaming** - Real-time data flow for long operations

In [ ]:
# Let's verify your Python environment is ready for MCP development
import sys
print(f"Python version: {sys.version}")
print(f"Python path: {sys.executable}")

# Check minimum version (3.10+ recommended for MCP)
assert sys.version_info >= (3, 10), "MCP requires Python 3.10 or higher"
print("\n✅ Python version is compatible with MCP!")

## 4. MCP Architecture: The Client-Host-Server Triad

Unlike traditional client-server models, MCP introduces a **tripartite architecture**:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP ARCHITECTURAL TRIAD                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │                         HOST PROCESS                             │    │
│  │        (Claude Desktop, Cursor IDE, Custom Application)          │    │
│  │                                                                   │    │
│  │  ┌─────────────────────────────────────────────────────────┐    │    │
│  │  │                    LLM / AI Model                        │    │    │
│  │  │              (Reasoning & Decision Making)               │    │    │
│  │  └─────────────────────────────────────────────────────────┘    │    │
│  │                              │                                   │    │
│  │                              ▼                                   │    │
│  │  ┌───────────────┐   ┌───────────────┐   ┌───────────────┐     │    │
│  │  │  MCP Client   │   │  MCP Client   │   │  MCP Client   │     │    │
│  │  │  Instance 1   │   │  Instance 2   │   │  Instance 3   │     │    │
│  │  └───────┬───────┘   └───────┬───────┘   └───────┬───────┘     │    │
│  │          │                   │                   │              │    │
│  └──────────┼───────────────────┼───────────────────┼──────────────┘    │
│             │                   │                   │                   │
│             │ 1:1 Connection    │ 1:1 Connection    │ 1:1 Connection   │
│             │                   │                   │                   │
│             ▼                   ▼                   ▼                   │
│  ┌───────────────┐   ┌───────────────┐   ┌───────────────┐            │
│  │  MCP Server   │   │  MCP Server   │   │  MCP Server   │            │
│  │  (Postgres)   │   │   (Slack)     │   │ (Filesystem)  │            │
│  └───────────────┘   └───────────────┘   └───────────────┘            │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Component Responsibilities

#### 1. The Host Process

The **Host** is the container and governance authority:

| Responsibility | Description |
|----------------|-------------|
| Lifecycle Management | Spawns/terminates client instances |
| Security Policy | Decides which servers are allowed |
| Context Aggregation | Combines data from multiple servers |
| User Interface | Obtains user consent for actions |
| Human-in-the-Loop | Ensures user approval for sensitive operations |

**Examples:** Claude Desktop, Cursor IDE, VS Code, Custom Python apps

#### 2. The MCP Client

The **Client** maintains a 1:1 stateful connection with a server:

| Responsibility | Description |
|----------------|-------------|
| Protocol Translation | Converts LLM intents to JSON-RPC |
| Handshake | Negotiates versions and capabilities |
| Session Management | Maintains connection state |
| Request/Response | Handles async communication |

#### 3. The MCP Server

The **Server** wraps specific data sources or tools:

| Responsibility | Description |
|----------------|-------------|
| Schema Definition | Defines tool/resource schemas |
| Request Execution | Processes and executes requests |
| Result Formatting | Returns structured responses |
| Error Handling | Reports failures semantically |

**Unix Philosophy:** Optimal MCP servers are **modular and single-purpose**.

In [ ]:
# Conceptual representation of MCP components
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
from enum import Enum

class ComponentRole(Enum):
    HOST = "host"
    CLIENT = "client"
    SERVER = "server"

@dataclass
class MCPComponent:
    """Represents a component in the MCP architecture."""
    name: str
    role: ComponentRole
    responsibilities: List[str]
    
    def describe(self) -> str:
        resp_str = "\n    - ".join(self.responsibilities)
        return f"""
MCP {self.role.value.upper()}: {self.name}
  Responsibilities:
    - {resp_str}
"""

# Define the three components
host = MCPComponent(
    name="Claude Desktop",
    role=ComponentRole.HOST,
    responsibilities=[
        "Lifecycle management of client instances",
        "Security policy enforcement",
        "Context aggregation from multiple servers",
        "User interface and consent management"
    ]
)

client = MCPComponent(
    name="Postgres Client Instance",
    role=ComponentRole.CLIENT,
    responsibilities=[
        "Maintain 1:1 connection with Postgres server",
        "Translate LLM intents to JSON-RPC messages",
        "Handle capability negotiation",
        "Manage session state"
    ]
)

server = MCPComponent(
    name="Postgres MCP Server",
    role=ComponentRole.SERVER,
    responsibilities=[
        "Define query/execute tool schemas",
        "Execute SQL against database",
        "Return structured results",
        "Handle errors with semantic codes"
    ]
)

# Display the architecture
print("=" * 60)
print("MCP ARCHITECTURAL COMPONENTS")
print("=" * 60)
for component in [host, client, server]:
    print(component.describe())

## 5. MCP Primitives: Tools, Resources, and Prompts

MCP defines three fundamental types of context a server can provide - the **Three Pillars of MCP**:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    THE THREE PILLARS OF MCP                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐        │
│   │                 │  │                 │  │                 │        │
│   │     TOOLS       │  │   RESOURCES     │  │    PROMPTS      │        │
│   │                 │  │                 │  │                 │        │
│   │   🔧 Action     │  │   👁️ Perception │  │   📋 Guidance   │        │
│   │   "The Hands"   │  │   "The Eyes"    │  │"The Instructions"│       │
│   │                 │  │                 │  │                 │        │
│   ├─────────────────┤  ├─────────────────┤  ├─────────────────┤        │
│   │                 │  │                 │  │                 │        │
│   │ • Execute funcs │  │ • Read-only data│  │ • Templates     │        │
│   │ • Modify state  │  │ • URI-based     │  │ • Best practices│        │
│   │ • Call APIs     │  │ • Subscribable  │  │ • Workflows     │        │
│   │                 │  │                 │  │                 │        │
│   ├─────────────────┤  ├─────────────────┤  ├─────────────────┤        │
│   │                 │  │                 │  │                 │        │
│   │ Model-Controlled│  │ App-Controlled  │  │ User-Controlled │        │
│   │                 │  │                 │  │                 │        │
│   └─────────────────┘  └─────────────────┘  └─────────────────┘        │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### 5.1 Tools (Action - "The Hands")

**Tools** are executable functions that allow the model to interact with the world:

| Aspect | Description |
|--------|-------------|
| Purpose | Execute actions, modify state, call APIs |
| Definition | JSON Schema for inputs/outputs |
| Control | **Model-controlled** - AI decides when to call |
| Methods | `tools/list`, `tools/call` |

**Examples:**
- `searchFlights(origin, destination, date)` - Search for flights
- `sendEmail(to, subject, body)` - Send an email
- `executeQuery(sql)` - Run a database query
- `createFile(path, content)` - Create a file

In [ ]:
# Example: Tool definition structure
import json

# This is what a tool definition looks like in MCP
tool_definition = {
    "name": "searchFlights",
    "description": "Search for available flights between two airports",
    "inputSchema": {
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "description": "Origin airport code (e.g., 'JFK')"
            },
            "destination": {
                "type": "string",
                "description": "Destination airport code (e.g., 'LAX')"
            },
            "date": {
                "type": "string",
                "format": "date",
                "description": "Travel date (YYYY-MM-DD)"
            },
            "passengers": {
                "type": "integer",
                "default": 1,
                "description": "Number of passengers"
            }
        },
        "required": ["origin", "destination", "date"]
    }
}

print("Example MCP Tool Definition:")
print("=" * 50)
print(json.dumps(tool_definition, indent=2))

### 5.2 Resources (Perception - "The Eyes")

**Resources** are passive, read-only data sources providing contextual information:

| Aspect | Description |
|--------|-------------|
| Purpose | Provide context, facts, documents |
| Identification | URI-based (e.g., `file:///path` or custom schemes) |
| Control | **Application-controlled** - User/app chooses what to fetch |
| Methods | `resources/list`, `resources/read`, `resources/subscribe` |

**Types:**
1. **Direct Resources** - Fixed URIs to specific data
2. **Resource Templates** - Parameterized URIs for queries

**Examples:**
- `file:///home/user/report.pdf` - A specific file
- `calendar://events/2024` - Calendar events
- `weather://forecast/{city}/{date}` - Weather template

In [ ]:
# Example: Resource definitions

# Direct resource
direct_resource = {
    "uri": "file:///documents/quarterly-report.pdf",
    "name": "Q4 2024 Financial Report",
    "description": "Company quarterly financial report",
    "mimeType": "application/pdf"
}

# Resource template (parameterized)
resource_template = {
    "uriTemplate": "weather://forecast/{city}/{date}",
    "name": "Weather Forecast",
    "description": "Get weather forecast for a city on a specific date",
    "mimeType": "application/json"
}

print("Direct Resource:")
print(json.dumps(direct_resource, indent=2))
print("\nResource Template:")
print(json.dumps(resource_template, indent=2))

### 5.3 Prompts (Guidance - "The Instructions")

**Prompts** are predefined templates that guide model behavior:

| Aspect | Description |
|--------|-------------|
| Purpose | Structure interactions, encode best practices |
| Definition | Templates with placeholders for arguments |
| Control | **User-controlled** - Explicitly selected by user |
| Methods | `prompts/list`, `prompts/get` |

**Examples:**
- "Plan a vacation" - Multi-step travel planning template
- "Security Audit" - Code review with security criteria
- "Summarize Meeting" - Meeting notes summarization format

In [ ]:
# Example: Prompt definition

prompt_definition = {
    "name": "plan-vacation",
    "description": "Plan a vacation with flights, hotels, and activities",
    "arguments": [
        {
            "name": "destination",
            "description": "Where do you want to go?",
            "required": True
        },
        {
            "name": "dates",
            "description": "Travel dates (start - end)",
            "required": True
        },
        {
            "name": "budget",
            "description": "Total budget in USD",
            "required": False
        }
    ]
}

print("Example MCP Prompt Definition:")
print("=" * 50)
print(json.dumps(prompt_definition, indent=2))

### Primitives Summary Table

| Primitive | Cognitive Function | Control | MCP Methods | Examples |
|-----------|-------------------|---------|-------------|----------|
| **Tools** | Action ("Hands") | Model-controlled | `tools/list`, `tools/call` | Search, Send, Execute |
| **Resources** | Perception ("Eyes") | App-controlled | `resources/list`, `resources/read` | Files, DB rows, APIs |
| **Prompts** | Guidance ("Instructions") | User-controlled | `prompts/list`, `prompts/get` | Templates, Workflows |

## 6. Transport Mechanisms: STDIO vs HTTP

MCP supports two distinct transport mechanisms:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP TRANSPORT MECHANISMS                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   STDIO (Standard I/O)                HTTP + SSE                        │
│   ════════════════════                ══════════                        │
│                                                                          │
│   ┌──────────┐                        ┌──────────┐                      │
│   │   Host   │                        │   Host   │                      │
│   │ Process  │                        │ Process  │                      │
│   └────┬─────┘                        └────┬─────┘                      │
│        │                                   │                            │
│        │ stdin/stdout                      │ HTTP POST                  │
│        │ (subprocess)                      │ + SSE responses            │
│        │                                   │                            │
│        ▼                                   ▼                            │
│   ┌──────────┐                        ════════════                      │
│   │  Local   │                           Network                        │
│   │  Server  │                        ════════════                      │
│   │ (same    │                             │                            │
│   │ machine) │                             ▼                            │
│   └──────────┘                        ┌──────────┐                      │
│                                       │  Remote  │                      │
│                                       │  Server  │                      │
│   ✅ No network overhead              │ (cloud)  │                      │
│   ✅ Secure by default                └──────────┘                      │
│   ✅ Simple setup                                                       │
│   ❌ Single user only                 ✅ Multi-client support           │
│   ❌ Not shareable                    ✅ Scalable                       │
│                                       ✅ Remote access                  │
│                                       ❌ Auth complexity                │
│                                       ❌ Network latency                │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Transport Comparison

| Aspect | STDIO | HTTP + SSE |
|--------|-------|------------|
| **Use Case** | Local tools, development | Remote/shared services |
| **Security** | Inherits process permissions | Requires auth (OAuth, tokens) |
| **Scalability** | Single user | Multiple clients |
| **Setup** | Simple (subprocess) | Complex (network, auth) |
| **Latency** | Minimal | Network-dependent |
| **Examples** | Local filesystem, dev tools | Cloud databases, enterprise APIs |

We'll dive deep into each transport in **Notebooks 3 and 4**.

## 7. The MCP Lifecycle

MCP is **stateful** - connections follow a defined lifecycle:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP CONNECTION LIFECYCLE                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   CLIENT                                              SERVER            │
│   ══════                                              ══════            │
│                                                                          │
│      │                                                   │              │
│      │  1. initialize (version, capabilities)           │              │
│      │ ─────────────────────────────────────────────────►│              │
│      │                                                   │              │
│      │  2. initialize response (server info, caps)       │              │
│      │ ◄─────────────────────────────────────────────────│              │
│      │                                                   │              │
│      │  3. initialized notification                      │              │
│      │ ─────────────────────────────────────────────────►│              │
│      │                                                   │              │
│      │         ╔════════════════════════════╗           │              │
│      │         ║   SESSION ESTABLISHED      ║           │              │
│      │         ║   (Normal Operations)      ║           │              │
│      │         ╚════════════════════════════╝           │              │
│      │                                                   │              │
│      │  4. tools/list, resources/read, etc.             │              │
│      │ ◄────────────────────────────────────────────────►│              │
│      │                                                   │              │
│      │  5. shutdown / connection close                   │              │
│      │ ─────────────────────────────────────────────────►│              │
│      │                                                   │              │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Example: MCP Initialization handshake (conceptual)

# Client sends initialize request
init_request = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": "2025-06-18",
        "capabilities": {
            "tools": {"listChanged": True},
            "resources": {"subscribe": True}
        },
        "clientInfo": {
            "name": "MyAIApp",
            "version": "1.0.0"
        }
    }
}

# Server responds with its capabilities
init_response = {
    "jsonrpc": "2.0",
    "id": 1,
    "result": {
        "protocolVersion": "2025-06-18",
        "capabilities": {
            "tools": {"listChanged": True},
            "resources": {"subscribe": True, "listChanged": True}
        },
        "serverInfo": {
            "name": "PostgresMCPServer",
            "version": "2.1.0"
        }
    }
}

print("1. Client Initialize Request:")
print(json.dumps(init_request, indent=2))
print("\n2. Server Initialize Response:")
print(json.dumps(init_response, indent=2))

## 8. Setting Up Your Development Environment

Let's prepare your Windows environment for MCP development.

In [ ]:
# Step 1: Check Python and pip
import subprocess
import sys

print("Environment Check")
print("=" * 50)
print(f"Python: {sys.version}")
print(f"Executable: {sys.executable}")

### Required Packages

Run these commands in your **PowerShell** to install MCP dependencies:

```powershell
# Create a virtual environment (recommended)
python -m venv mcp-env
.\mcp-env\Scripts\Activate.ps1

# Install core MCP packages
pip install mcp fastmcp

# Install additional useful packages
pip install pydantic httpx aiohttp

# For later notebooks (framework integration)
pip install langchain-mcp-adapters crewai
```

In [ ]:
# Verify installations (run after installing packages)
def check_package(package_name):
    try:
        __import__(package_name.replace("-", "_"))
        return "✅ Installed"
    except ImportError:
        return "❌ Not installed"

packages = ["mcp", "fastmcp", "pydantic", "httpx", "aiohttp"]

print("Package Status")
print("=" * 40)
for pkg in packages:
    status = check_package(pkg)
    print(f"{pkg:20} {status}")

## 9. Key Takeaways

### What We Covered

1. **The M×N Problem** - MCP reduces integration complexity from M×N to M+N
2. **MCP Architecture** - Host → Client → Server tripartite model
3. **Three Pillars** - Tools (actions), Resources (data), Prompts (guidance)
4. **Transport Options** - STDIO for local, HTTP+SSE for remote
5. **Lifecycle** - Stateful sessions with capability negotiation

### MCP in One Sentence

> **MCP is the "USB-C for AI"** - a universal protocol that lets any AI model connect to any tool or data source through a standardized, secure interface.

### Next Steps

In **Notebook 2: MCP Primitives Deep Dive**, we'll:
- Build actual Tools, Resources, and Prompts
- Understand JSON Schema for tool definitions
- Explore resource subscriptions
- Create prompt templates

## 10. Exercises

### Exercise 1: Identify the Components

For the following scenario, identify the Host, Client, and Server:

*"A developer uses VS Code with an AI assistant that can query their PostgreSQL database."*

- **Host:** ?
- **Client:** ?
- **Server:** ?

In [ ]:
# Exercise 1: Fill in your answers
exercise_1 = {
    "host": "___",  # Your answer here
    "client": "___",  # Your answer here  
    "server": "___"  # Your answer here
}

# Uncomment to check your answer
# answer_1 = {
#     "host": "VS Code",
#     "client": "MCP Client instance for PostgreSQL",
#     "server": "PostgreSQL MCP Server"
# }
# print(answer_1)

### Exercise 2: Classify the Primitive

For each capability below, identify if it's a **Tool**, **Resource**, or **Prompt**:

1. `sendSlackMessage(channel, message)` - ?
2. `file:///config/settings.json` - ?
3. "Code Review Template" - ?
4. `executeSQL(query)` - ?
5. `calendar://events/today` - ?

In [ ]:
# Exercise 2: Fill in your answers ("Tool", "Resource", or "Prompt")
exercise_2 = {
    "sendSlackMessage": "___",
    "file:///config/settings.json": "___",
    "Code Review Template": "___",
    "executeSQL": "___",
    "calendar://events/today": "___"
}

# Uncomment to check your answers
# answer_2 = {
#     "sendSlackMessage": "Tool",
#     "file:///config/settings.json": "Resource",
#     "Code Review Template": "Prompt",
#     "executeSQL": "Tool",
#     "calendar://events/today": "Resource"
# }
# print(answer_2)

### Exercise 3: Design a Tool Schema

Create a JSON Schema for a tool that searches for books by title, author, or ISBN.

In [ ]:
# Exercise 3: Design your tool schema
book_search_tool = {
    "name": "searchBooks",
    "description": "___",  # Add description
    "inputSchema": {
        "type": "object",
        "properties": {
            # Add your properties here
        },
        "required": []  # Which fields are required?
    }
}

print(json.dumps(book_search_tool, indent=2))

---

## References & Further Reading

- [MCP Official Documentation](https://modelcontextprotocol.io/docs)
- [MCP Architecture Overview](https://modelcontextprotocol.io/docs/learn/architecture)
- [Anthropic MCP Announcement](https://www.anthropic.com/news/model-context-protocol)
- [MCP GitHub Repository](https://github.com/anthropics/mcp)
- [Building Effective AI Agents - Anthropic](https://www.anthropic.com/research/building-effective-agents)

---

**Next Notebook:** [02_MCP_Primitives_Deep_Dive.ipynb](./02_MCP_Primitives_Deep_Dive.ipynb)